In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import skimage.io as io
from skimage import data_dir
import pandas as pd
import split_folders
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from skimage.io import imread_collection
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import random

In [4]:
#this function reads in images and then flattens them into a dataframe
def flat_images(path):
    images=io.ImageCollection(path)
    flat_images=[]
    for i in range(len(images)):
        img=images[i]
        img1=img.flatten()
        flat_images.append(img1)
    df=pd.DataFrame(flat_images)
    return df

In [5]:
#flattens ditylum images
path= r'dit\*.tif'
df_1=flat_images(path)
df_1=df_1.fillna(0)
df_1['label']=1

In [6]:
#flattens non-ditylum images
#path1= r'C:\Users\vrm8601\Documents\Pre Prac\0_notditylum\0\*.tif'
path1= r'not\*.tif'
df_0=flat_images(path1)
df_0=df_0.fillna(0)
df_0['label']=0

In [7]:
#puts both sets of images in same dataframe
df=df_1.append(df_0)
df.reset_index(inplace=True,drop=True)
df=df.fillna(0)

In [9]:
# create test and train data (80/20 split)
X=df.drop(['label'],axis=1)
y=df.label
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:

scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train_transform = scaler.transform(X_train)
X_test_transform = scaler.transform(X_test)
# perform PCA
pca = PCA(.99)
pca.fit(X_train_transform)
X_train_transform = pca.transform(X_train_transform)
X_test_transform = pca.transform(X_test_transform)

In [12]:
#params={'C': np.linspace(0,1,5)}
log_reg = LogisticRegression(random_state=42,solver='lbfgs',max_iter=50000)

log = GridSearchCV(log_reg,params,cv=3, scoring = 'accuracy')
log.fit(X_train_transform,y_train)
print("best C",log.best_params_,"bestscore",log.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:943: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1. / C, sample_weight),
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:119: RuntimeWarning: invalid value encountered in double_scalars
  out = -np.sum(sample_weight * log_logistic(yz)) + .5 * alpha * np.dot(w, w)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:124: RuntimeWarning: invalid value encountered in multiply
  grad[:n_features] = safe_sparse_dot(X.T, z0) + alpha * w
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
log_reg = LogisticRegression(random_state=42,solver='lbfgs',max_iter=50000, C=1000)
log_reg.fit(X_train_transform,y_train)
y_pred=logReg.predict(X_test_transform)

# metrics
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))